In [1]:
%reload_ext jupyter_black
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys

project_root = os.path.abspath("..")
os.chdir(project_root)
sys.path.append(project_root)

In [8]:
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import json
import re
import torch

In [9]:
df = pd.concat(
    [pd.read_csv('dataset/movie_quotes_sometimes_correct.csv'), pd.read_csv('dataset/often_hallucinating.csv')],
    ignore_index=True,
)
df['correct_ratio'] = df['correct_ratio'].fillna(0)
df

,quote,title,correct_ratio,attempted_ratio,positive_probability
0,I live my life a quarter mile at a time.,The Fast and the Furious,0.4,1.0,1.000000
1,You only live twice:Once when you are born And...,You only live twice,0.3,1.0,1.000000
2,Ask any racer. Any real racer. It don't matter...,The Fast and The Furious,0.1,1.0,0.999976
3,The greatest thing you’ll ever learn is just t...,Moulin Rouge,0.1,1.0,0.999805
4,"I remember the story of an evil witch, at the ...",Maleficent,0.5,1.0,0.999490
5,"Well, I’ve got no leashes, or fences. With me,...",The Lady and the Tramp,0.1,0.9,0.964120
6,An idea is like a virus. Resilient. Highly con...,Inception,0.3,0.9,0.939586
7,It's not about what I want. It's about what's...,The Dark Knight,0.1,0.9,0.898785
8,Leave One Wolf Alive and the Sheep are Never Safe,Game of Thrones Season 7,0.1,0.6,0.730845
9,"When all seems lost, a few brave souls can sav...",Transformers,0.1,0.4,0.453989


In [10]:
df_confidence_steered = pd.read_csv('dataset/confidence_steering_output_checking.csv')
df_confidence_steered

,quote,title,side,correct_ratio,attempted_ratio,positive_probability
0,Ask any racer. Any real racer. It don't matter...,The Fast and The Furious,confident,0.3,1.0,0.999949
1,"When all seems lost, a few brave souls can sav...",Transformers,confident,0.0,1.0,0.999901
2,You used my social security number to open sev...,NaN,confident,0.0,1.0,0.999870
3,My childhood dog! You adopted him from the she...,NaN,confident,0.0,1.0,0.999870
4,"Well, I’ve got no leashes, or fences. With me,...",The Lady and the Tramp,confident,0.0,1.0,0.999850
5,My son's wedding photos! You burned every sing...,NaN,confident,0.0,1.0,0.999802
6,My entire life savings! You gambled away two h...,NaN,confident,0.0,1.0,0.999797
7,My brother's Purple Heart! You pawned a dead M...,NaN,confident,0.0,1.0,0.999773
8,"The prosecutor has twelve witnesses, security ...",NaN,confident,0.0,1.0,0.999712
9,The priest said you confessed to molesting alt...,NaN,confident,0.0,1.0,0.999687
